# Tutorial 2
Done by Chensheng LUO of CentraleSupélec

We first load the data needed.

In [3]:
array = []
 
for line in open('Projectdata.txt'):
    array.extend(line.strip().split(' '))
 
array = map(float, array)
print(list(array))

[98474.4, 12373.4, 107318.4, 9739.7, 13000.1, 12207.7, 63589.4, 31893.8, 5784.9, 9662.7, 61731.1, 15269.2, 4730.0, 11269.2, 26947.1, 27838.1, 90682.5, 8086.6, 7905.6, 48162.7]
